# Capstone Project - The Battle of Neighborhoods

## The best place for an Italian Restaurant in Rome

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

#### A description of the problem and a discussion of the background.
Rome is a city rich in history and monuments. It is one of the cities with the greatest tourist attraction.
Consequently, it has a large presence of tourists and therefore of potential customers during all the year.

The opening of an Italian restaurant in Rome, therefore, represents a possible highly profitable activity.
Now, the question is: where would be the best place they should open it?

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **Rome**, Italy.

Since there are lots of restaurants in Rome we will try to detect **locations that are not already crowded with italian restaurants**. We are also particularly interested in **areas close to the city center**.
The proximity to the center of the city could represent a determining factor given the greater number of tourists.

## Data <a name="data"></a>
#### A description of the data and how it will be used to solve the problem.

The data I am going to use to make this study are different. First of all, I will need a map of Rome.

Rome is divided into municipalities that have their own local administration (https://it.wikipedia.org/wiki/Municipi_di_Roma). 
I can have a list of all the municipalities and their geografical limits in the geojson file **municipi.geojson** from the web page: https://geonue.com/mappa-interattiva-municipi-di-roma/

I decide to study the Central Municipality (Municipio I- Rome Center) because it is the historical heart of the city. It can be divided in sub-regions called **rioni**.

A **rione of Rome** is a traditional administrative division of the city of Rome. "Rione" is an Italian term used since the 14th century to name a district of a town. The term was born in Rome, originating from the administrative divisions of the city. The word comes from the Latin word regio (pl. regiones, meaning region); during the Middle Ages the Latin word became rejones, from which rione comes. Currently, all the rioni are located in Municipio I of Rome. (https://it.wikipedia.org/wiki/Rioni_di_Roma)

Then, I obtained a list of all the rioni from Wikipedia. I obtained the approximate position of each rione in the Municipio I with Google Maps.

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the area (any type of restaurant)
* number of Italian restaurants in the area
* distance from the city center

Number of restaurants and their type and location in every rione will be obtained using **Foursquare API**.


### Neighborhood Candidates

 
Let's first find the latitude & longitude of Rome city center, using geopy.

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

Take the geographical coordinate of Rome

In [5]:
address = 'Rome, Italy'

geolocator = Nominatim(user_agent="Rome_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.8933203 12.4829321


Create map of Rome using latitude and longitude values

In [6]:
map_Rome = folium.Map(location=[latitude,longitude], zoom_start=10)    
map_Rome

Let's plot the limits of the municipalities of Rome. 

In [7]:
municipi_geo = r'dati/municipi.geojson' # geojson file

# create a plain world map
map_Rome = folium.Map(location=[latitude,longitude], zoom_start=11)
map_Rome.choropleth(
    geo_data=municipi_geo,
    key_on='feature.properties.municipio',
    fill_color='#3186cc', 
    fill_opacity=0.2, 
    line_opacity=1,
    reset=True
)
map_Rome

I decide to study the Central Municipality (**Municipio I - Rome Center**) because it is the historical heart of the city.

In [11]:
import geojson
with open("dati/municipi.geojson") as f:
    gj = geojson.load(f)
features = gj['features'][0]

map_Rome_center = folium.Map(location=[latitude,longitude], zoom_start=11)
map_Rome_center.choropleth(
    geo_data=features,
    key_on='feature.properties.municipio',
    fill_opacity=0.1,
    line_opacity=1,
    reset=True
)
map_Rome_center

**Municipio I** can be divided in **22** sub-regions called **rioni**.

A **rione** of Rome is a traditional administrative division of the city of Rome. Currently, all the **rioni** are located in Municipio I of Rome (https://it.wikipedia.org/wiki/Rioni_di_Roma).

Let's create a list of all **rioni** of Rome from Wikipedia using **BeautifulSoup**

In [13]:
import requests
from bs4 import BeautifulSoup

URL = "https://it.wikipedia.org/wiki/Rioni_di_Roma"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

In [14]:
link = My_table.findAll("a")

rione = []
for l in link:
    rione.append(l.get("title"))

df = pd.DataFrame()
df["Rioni"] = rione
df = df.dropna()
df = df[df["Rioni"]!= "Municipio Roma I"]
df = df.reset_index(drop=True)
df = df.replace(to_replace='\(rione di Roma\)', value='', regex=True)
df

,Rioni
0,Monti
1,Trevi
2,Colonna
3,Campo Marzio
4,Ponte
5,Parione
6,Regola
7,Sant'Eustachio
8,Pigna
9,Campitelli


Let's create a list of latitude and longitude of each **rione**. The coordinates are taken manually from GoogleMaps. Then I add this info to the DataFrame.

In [16]:
lat_lon_rioni = [[41.894962, 12.489225],
                [41.902773, 12.485952],
                [41.902222, 12.477778],
                [41.908458, 12.47703],
                [41.90031, 12.465617],
                [41.897599, 12.471746],
                [41.894722, 12.471111],
                [41.898333, 12.475556],
                [41.9, 12.483333],
                [41.88803, 12.486391],
                [41.893333, 12.477778],
                [41.885346, 12.481799],
                [41.887222, 12.465556],
                [41.904066, 12.461629],
                [41.893056, 12.503889],
                [41.907164, 12.490854],
                [41.908078, 12.465706],
                [41.9043887, 12.501916],
                [41.88145, 12.494751],
                [41.876111, 12.475],
                [41.879017, 12.486293],
                [41.908078, 12.465706]]
df[["Latitude", "Longitude"]] = pd.DataFrame(lat_lon_rioni)
df

,Rioni,Latitude,Longitude
0,Monti,41.894962,12.489225
1,Trevi,41.902773,12.485952
2,Colonna,41.902222,12.477778
3,Campo Marzio,41.908458,12.477030
4,Ponte,41.900310,12.465617
5,Parione,41.897599,12.471746
6,Regola,41.894722,12.471111
7,Sant'Eustachio,41.898333,12.475556
8,Pigna,41.900000,12.483333
9,Campitelli,41.888030,12.486391


Let's plot the positions of **rioni** on the map inside the **Municipio I** area.

In [18]:
import geojson
with open("dati/municipi.geojson") as f:
    gj = geojson.load(f)
features = gj['features'][0]

map_Rome_center = folium.Map(location=[latitude,longitude], zoom_start=13)
map_Rome_center.choropleth(
    geo_data=features,
    key_on='feature.properties.municipio',
    fill_opacity=0.1,
    line_opacity=1,
    reset=True
)
map_Rome_center
for lat, lon in zip(df.Latitude, df.Longitude):
    folium.CircleMarker([lat,lon],
        radius=5,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=1,
        parse_html=False).add_to(map_Rome_center)  

map_Rome_center

OK, we now have the coordinates of **rioni**/areas to be evaluated. 


### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each area.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on Italian restaurants in the area.

Foursquare credentials are defined in the cell bellow.

In [19]:
CLIENT_ID = '5L1YQGKVMRRQASTMKOZ3N2NZZ31QSFDWWPB04DAKZLHCDFXI' # your Foursquare ID
CLIENT_SECRET = 'LXUHWGJPDJNWFZPDYFMNXDGVOGONJU1PAHJVYRCVGF43PZLI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5L1YQGKVMRRQASTMKOZ3N2NZZ31QSFDWWPB04DAKZLHCDFXI
CLIENT_SECRET:LXUHWGJPDJNWFZPDYFMNXDGVOGONJU1PAHJVYRCVGF43PZLI


In [20]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Italy', '')
    #address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=1000, limit=300):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [21]:
# Let's now go over our rioni locations and get nearby restaurants. 
# We'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=500, limit=300)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                #x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian)
                if venue_distance<=500:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_500.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_500.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_500.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(df.Latitude, df.Longitude)
    
    # Let's persists this in local file system
    with open('restaurants_500.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_500.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_500.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [23]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 616
Total number of Italian restaurants: 445
Percentage of Italian restaurants: 72.24%
Average number of restaurants in neighborhood: 39.59090909090909


In [24]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('53f8fe8c498ecb81e43b7c38', 'Temakinho', 41.89548557417076, 12.490374056068802, 'Via dei Serpenti, 16, 00184 Roma Lazio, Italia', 111, False)
('4daf45b55da3f2f3d2a2673f', 'Ristorante Tipico "Tavernelle"', 41.896354, 12.490539, 'Via Panisperna 48 (Via del Boschetto), 00184 Roma Lazio, Italia', 189, True)
('57409fca498e93d605dcd51f', 'Pastachef', 41.894749, 12.489673, 'Via Baccina, 42, 00184 Roma Lazio, Italia', 44, True)
('4ba7e9fbf964a52038be39e3', 'La Taverna dei Fori Imperiali', 41.89398899985174, 12.488527714599032, 'Via della Madonna dei Monti, 9 (Via Cavour), 00184 Roma Lazio, Italia', 122, True)
('58dc0effaf5c147525a6a093', 'Osteria Oliva', 41.896604, 12.489945, '00184 Roma Lazio, Italia', 192, True)
('4adcdacff964a520975521e3', 'Angelino "ai Fori" dal 1947', 41.893154328120836, 12.48796839836952, 'Largo Corrado Ricci, 40/43a (Via dei Fori Imperiali), 00184 Roma Lazio, Italia', 226, True)
('55990b6a498e7e81846b533f', 'All Bi One', 

In [25]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('4daf45b55da3f2f3d2a2673f', 'Ristorante Tipico "Tavernelle"', 41.896354, 12.490539, 'Via Panisperna 48 (Via del Boschetto), 00184 Roma Lazio, Italia', 189, True)
('57409fca498e93d605dcd51f', 'Pastachef', 41.894749, 12.489673, 'Via Baccina, 42, 00184 Roma Lazio, Italia', 44, True)
('4ba7e9fbf964a52038be39e3', 'La Taverna dei Fori Imperiali', 41.89398899985174, 12.488527714599032, 'Via della Madonna dei Monti, 9 (Via Cavour), 00184 Roma Lazio, Italia', 122, True)
('58dc0effaf5c147525a6a093', 'Osteria Oliva', 41.896604, 12.489945, '00184 Roma Lazio, Italia', 192, True)
('4adcdacff964a520975521e3', 'Angelino "ai Fori" dal 1947', 41.893154328120836, 12.48796839836952, 'Largo Corrado Ricci, 40/43a (Via dei Fori Imperiali), 00184 Roma Lazio, Italia', 226, True)
('4bddf106e75c0f47c43ac603', 'La Base', 41.89388456581016, 12.490003885915206, 'Via Cavour, 276, 00184 Roma Lazio, Italia', 136, True)
('4df8f4e7d22dca97c599e87b', 'Le Tavernelle

In [26]:

print('Restaurants around location')
print('---------------------------')
#location_restaurants[22][0]
for i in range(0, 22):
    rs = location_restaurants[i][:]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {} \n'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 1: Temakinho, Ristorante Tipico "Tavernelle", Pastachef, La Taverna dei Fori Imperiali, Osteria Oliva, Angelino "ai Fori" dal 1947, All Bi One, La Base, Le Tavernelle, Suburra 1930, Maharajah, Madre, Mariolina, Ristorante Valentino, Maccheroni Express, Pasta Imperiale, Ristorante Roof Garden Forum Roma, Dodo Cucina & Sapori, Tin House, Sitar, Vecchia Roma, Taverna Romana, Da Valentino, Iari The Vino, Il Colonne Ristorante Pizzeria, Gli Angeletti, Sciué Sciué Cucina & Caffè, L'Archetto di Cavour, Due Colonne, L'Asino D'oro, Cuoco & Camicia, La Taverna dei Monti, Doozo, Hosteria La Vacca 'Mbriaca, Le Lanterne, Hostaria Al Boschetto, Ristorante Tema, Sa Tanca Crostaceria, Trattoria Monti Il Girasole, Rinaldi al Quirinale, Indian Affairs, Matermatuta, Grotta D'Abruzzo, Caffetteria Ristorante Diadema, Il Grillo Brillo, Hostaria Perdingianu e Croccoriga, Ristorante Al Boschetto 

Restaurants around location 2

Let's now see all the collected restaurants in our area of interest on map, and let's also show Italian restaurants in different color.

    Italian restaurants are depicted in **red**
    Not-Italian restaurants are depicted in **blue**

In [27]:
folium.Marker([latitude, longitude]).add_to(map_Rome_center)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_Rome_center)
map_Rome_center

Looking good. So now we have all the restaurants in area, and we know which ones are Italian restaurants (red)! We also know which restaurants exactly are in vicinity of every rione candidate center (yellow).

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Rome that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area around city center.

In first step we have collected the required **data: location and type (category) of every restaurant**. We have also **identified Italian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Rome - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* low number of Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements**: we will take into consideration locations with 

    no more than 50 Italian restaurants in radius of 500 meters, 
    with a distance less then 1000 meters from the city Center.

We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones which should be a starting point for finalsearch for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. 
First let's count the **number of restaurants in every area candidate**:

In [28]:
location_restaurants_count = [len(res) for res in location_restaurants]

df['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=500m:', np.array(location_restaurants_count).mean())

df

Average number of restaurants in every area with radius=500m: 39.59090909090909


,Rioni,Latitude,Longitude,Restaurants in area
0,Monti,41.894962,12.489225,47
1,Trevi,41.902773,12.485952,65
2,Colonna,41.902222,12.477778,36
3,Campo Marzio,41.908458,12.477030,63
4,Ponte,41.900310,12.465617,63
5,Parione,41.897599,12.471746,74
6,Regola,41.894722,12.471111,55
7,Sant'Eustachio,41.898333,12.475556,68
8,Pigna,41.900000,12.483333,72
9,Campitelli,41.888030,12.486391,6


Second let's count the **number of Italina restaurants in every area candidate**:

In [30]:
ita_location_restaurants_count = []
for res in location_restaurants:
    count = 0
    for i in range(len(res)):
        is_italian = res[i][6]
        if is_italian: 
            count+=1
    ita_location_restaurants_count.append(count)
df['Italian Restaurants in area'] = ita_location_restaurants_count

print('Average number of italian restaurants in every area with radius=500m:', np.array(ita_location_restaurants_count).mean())

df

Average number of italian restaurants in every area with radius=500m: 28.727272727272727


,Rioni,Latitude,Longitude,Restaurants in area,Italian Restaurants in area
0,Monti,41.894962,12.489225,47,38
1,Trevi,41.902773,12.485952,65,51
2,Colonna,41.902222,12.477778,36,29
3,Campo Marzio,41.908458,12.477030,63,48
4,Ponte,41.900310,12.465617,63,51
5,Parione,41.897599,12.471746,74,57
6,Regola,41.894722,12.471111,55,37
7,Sant'Eustachio,41.898333,12.475556,68,53
8,Pigna,41.900000,12.483333,72,57
9,Campitelli,41.888030,12.486391,6,4


Let's crete a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. Also, let's show **borders of Municipio I** on our map.

In [31]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

italian_latlons = [[res[2], res[3]] for res in italian_restaurants.values()]

In [34]:
from folium import plugins
from folium.plugins import HeatMap

map_Rome_center2 = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_Rome_center2) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_Rome_center2)
folium.Marker([latitude, longitude]).add_to(map_Rome_center2)
map_Rome_center2.choropleth(
    geo_data=features,
    key_on='feature.properties.municipio',
    fill_opacity=0.1,
    line_opacity=1,
    reset=True
)
#folium.Circle([latitude, longitude], radius=1000, fill=False, color='white').add_to(map_Rome2)
#folium.Circle([latitude, longitude], radius=2000, fill=False, color='white').add_to(map_Rome2)
#folium.Circle([latitude, longitude], radius=3000, fill=False, color='white').add_to(map_Rome2)
#folium.GeoJson(berlin_boroughs, style_function=boroughs_style, name='geojson').add_to(map_Rome2)
map_Rome_center2

Looks like a few pockets of low restaurant density closest to city center can be found **south, south-east from city center**. 

Let's create another heatmap map showing **heatmap/density of Italian restaurants** only.

In [36]:
map_Rome_center_ita = folium.Map(location=[latitude,longitude], zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_Rome_center_ita) #cartodbpositron cartodbdark_matter
HeatMap(italian_latlons).add_to(map_Rome_center_ita)
folium.Marker([latitude, longitude]).add_to(map_Rome_center_ita)
map_Rome_center_ita.choropleth(
    geo_data=features,
    key_on='feature.properties.municipio',
    fill_opacity=0.1,
    line_opacity=1,
    reset=True
)
#folium.Circle(berlin_center, radius=1000, fill=False, color='white').add_to(map_Rome3)
#folium.Circle(berlin_center, radius=2000, fill=False, color='white').add_to(map_Rome3)
#folium.Circle(berlin_center, radius=3000, fill=False, color='white').add_to(map_Rome3)
#folium.GeoJson(berlin_boroughs, style_function=boroughs_style, name='geojson').add_to(map_Rome3)
map_Rome_center_ita

This map indicates higher density of existing Italian restaurants directly north, south and east from the center, with closest pockets of **low Italian restaurant density positioned south-east, south and south-west from city center**.

Based on this we will now focus our analysis on these areas.

In [37]:
df

,Rioni,Latitude,Longitude,Restaurants in area,Italian Restaurants in area
0,Monti,41.894962,12.489225,47,38
1,Trevi,41.902773,12.485952,65,51
2,Colonna,41.902222,12.477778,36,29
3,Campo Marzio,41.908458,12.477030,63,48
4,Ponte,41.900310,12.465617,63,51
5,Parione,41.897599,12.471746,74,57
6,Regola,41.894722,12.471111,55,37
7,Sant'Eustachio,41.898333,12.475556,68,53
8,Pigna,41.900000,12.483333,72,57
9,Campitelli,41.888030,12.486391,6,4


Let's calculate the distance of each **rione** from the center.

In [40]:
#calculate Distance Between Two Points on Earth given latitude and longitude
from math import radians, cos, sin, asin, sqrt 
def distance(lat1, lat2, lon1, lon2): 
	
	# The math module contains a function named 
	# radians which converts from degrees to radians. 
	lon1 = radians(lon1) 
	lon2 = radians(lon2) 
	lat1 = radians(lat1) 
	lat2 = radians(lat2) 
	
	# Haversine formula 
	dlon = lon2 - lon1 
	dlat = lat2 - lat1 
	a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

	c = 2 * asin(sqrt(a)) 
	
	# Radius of earth in kilometers. Use 3956 for miles 
	r = 6371
	
	# calculate the result 
	return(c * r) 

In [41]:
print("DISTANCE OF EACH RIONE FROM THE CENTER")

dist = []
for i,neigh in enumerate(df["Rioni"]):
    d = distance(latitude, df["Latitude"][i], longitude, df["Longitude"][i])
    print(df["Rioni"][i], d, "Km")
    dist.append(d)
df["Distance from Center"] = dist

DISTANCE OF EACH RIONE FROM THE CENTER
Monti  0.5519337903933887 Km
Trevi  1.0804016905120841 Km
Colonna  1.0778348066879342 Km
Campo Marzio 1.7526799671295423 Km
Ponte  1.630318632480701 Km
Parione 1.0409559509465711 Km
Regola  0.9907854240902176 Km
Sant'Eustachio  0.8266838325282626 Km
Pigna  0.7434895644160869 Km
Campitelli 0.6542312612179348 Km
Sant'Angelo  0.4266191330588701 Km
Ripa  0.8916487234105992 Km
Trastevere 1.5901602117965024 Km
Borgo  2.1298912491369855 Km
Esquilino  1.7349038515670478 Km
Ludovisi  1.6731598046129792 Km
Sallustiano 2.1737944136627267 Km
Castro Pretorio 1.995853622153776 Km
Celio  1.642981010598512 Km
Testaccio 2.0231163196193522 Km
San Saba  1.614605806405584 Km
Prati  2.1737944136627267 Km


In [42]:
df.head()

,Rioni,Latitude,Longitude,Restaurants in area,Italian Restaurants in area,Distance from Center
0,Monti,41.894962,12.489225,47,38,0.551934
1,Trevi,41.902773,12.485952,65,51,1.080402
2,Colonna,41.902222,12.477778,36,29,1.077835
3,Campo Marzio,41.908458,12.477030,63,48,1.752680
4,Ponte,41.900310,12.465617,63,51,1.630319


Let's sort our data by "Distance from Center" and "Italian Restaurants in area".

In [44]:
df.sort_values(by=["Distance from Center", 'Italian Restaurants in area'])

,Rioni,Latitude,Longitude,Restaurants in area,Italian Restaurants in area,Distance from Center
10,Sant'Angelo,41.893333,12.477778,29,19,0.426619
0,Monti,41.894962,12.489225,47,38,0.551934
9,Campitelli,41.888030,12.486391,6,4,0.654231
8,Pigna,41.900000,12.483333,72,57,0.743490
7,Sant'Eustachio,41.898333,12.475556,68,53,0.826684
11,Ripa,41.885346,12.481799,16,14,0.891649
6,Regola,41.894722,12.471111,55,37,0.990785
5,Parione,41.897599,12.471746,74,57,1.040956
2,Colonna,41.902222,12.477778,36,29,1.077835
1,Trevi,41.902773,12.485952,65,51,1.080402


The first lines represent possible candidates.
OK. Let us now **filter** those locations: we're interested only in **locations with no more than 50 italian restaurants in radius of 500 meters**, and **rioni with a distance from the city center len then 1000 meters**.

In [35]:
#rioni with less then 50 italian restaurants in a radius of 500 meters
df_short_list = df[df["Italian Restaurants in area"] <= 50].sort_values(by=["Distance from Center"])

#rioni with a distance that is less then 1km from the center of Rome
df_short_list = df_short_list[df_short_list["Distance from Center"]<=1]
df_short_list


,Rioni,Latitude,Longitude,Restaurants in area,Italian Restaurants in area,Distance from Center
10,Sant'Angelo,41.893333,12.477778,29,19,0.426619
0,Monti,41.894962,12.489225,47,38,0.551934
9,Campitelli,41.888030,12.486391,6,4,0.654231
11,Ripa,41.885346,12.481799,16,14,0.891649
6,Regola,41.894722,12.471111,55,37,0.990785


In [46]:
#for res in restaurants.values():
#    name = res[1]
#    lat = res[2]; lon = res[3]
#    is_italian = res[6]
#    print(name, lat, lon, is_italian)

#rest = pd.DataFrame(restaurants)
#rest = rest.transpose()
#rest.head()

OK. Let us now **filter** the locations we're interested in.

In [47]:
good_res_count = np.array(df["Italian Restaurants in area"] <= 50)
print('Locations with no more than 50 italian restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df["Distance from Center"]<=1)
print('Locations with distance from center less then 1Km:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df[good_locations]
df_good_locations

Locations with no more than 50 italian restaurants nearby: 17
Locations with distance from center less then 1Km: 7
Locations with both conditions met: 5


,Rioni,Latitude,Longitude,Restaurants in area,Italian Restaurants in area,Distance from Center
0,Monti,41.894962,12.489225,47,38,0.551934
6,Regola,41.894722,12.471111,55,37,0.990785
9,Campitelli,41.888030,12.486391,6,4,0.654231
10,Sant'Angelo,41.893333,12.477778,29,19,0.426619
11,Ripa,41.885346,12.481799,16,14,0.891649


Let's see how this looks on a map.

In [48]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_Rome_final = folium.Map(location=[latitude, longitude], zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(map_Rome_final)
HeatMap(restaurant_latlons).add_to(map_Rome_final)
#folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_berlin)
#folium.Marker(berlin_center).add_to(map_berlin)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=4, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Rome_final) 
#folium.GeoJson(berlin_boroughs, style_function=boroughs_style, name='geojson').add_to(map_berlin)
map_Rome_final

Looking good. We now have a bunch of locations fairly close to the center (less then 1km), and we know that each of those locations has no more than 50 italian restaurants in radius of 500m. Any of those locations is a potential candidate for a new Italian restaurant, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [49]:
map_Rome_final= folium.Map(location=[latitude, longitude], zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_Rome_final)
folium.Marker([latitude, longitude]).add_to(map_Rome_final)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Rome_final)
#folium.GeoJson(berlin_boroughs, style_function=boroughs_style, name='geojson').add_to(map_berlin)
map_Rome_final

What we have now is a clear indication of zones with low number of italian restaurants in vicinity.

Let us now **cluster** those locations to create **centers of zones containing good locations**. 

Those zones, their centers will be the final result of our analysis. 

In [51]:
from sklearn.cluster import KMeans

number_of_clusters = 3

good_xys = df_good_locations[['Latitude', 'Longitude']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)
kmeans.cluster_centers_

array([[41.886688 , 12.484095 ],
       [41.8940275, 12.4744445],
       [41.894962 , 12.489225 ]])

In [53]:
cluster_centers = kmeans.cluster_centers_

map_Rome_final = folium.Map(location=[latitude, longitude], zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_Rome_final)
HeatMap(restaurant_latlons).add_to(map_Rome_final)
folium.Circle([latitude, longitude], radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_Rome_final)
folium.Marker([latitude, longitude]).add_to(map_Rome_final)
for lat, lon in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_Rome_final) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Rome_final)
map_Rome_final

The clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the areas to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [57]:
map_Rome_final= folium.Map(location=[latitude, longitude], zoom_start=14)
folium.Marker([latitude, longitude]).add_to(map_Rome_final)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.1).add_to(map_Rome_final)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Rome_final)
for lat,lon in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_Rome_final) 
#folium.GeoJson(berlin_boroughs, style_function=boroughs_style, name='geojson').add_to(map_Rome_final)
map_Rome_final

Let's zoom in on candidates:

In [56]:
map_Rome_final = folium.Map(location=[41.889446 , 12.485805], zoom_start=15)
folium.Marker([latitude, longitude]).add_to(map_Rome_final)
for lat, lon in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_Rome_final) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.1).add_to(map_Rome_final)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Rome_final)
#folium.GeoJson(berlin_boroughs, style_function=boroughs_style, name='geojson').add_to(map_berlin)
map_Rome_final

This concludes our analysis. We have created 3 main areas representing centers of zones containing locations with low number of italian restaurants and all zones being fairly close to city center (all less than 1km from Rome center. 

Their centers/addresses should be considered only as a starting point for exploring area in search for potential restaurant locations. Most of the zones are located in rioni **Monti, Ripa, San'Angelo**, which we have identified as interesting due to being popular with tourists, fairly close to city center and well connected by public transport.

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Municipio I of Rome. It is the central municipality in Rome. 

After directing our attention to this more narrow area of interest we loooked to the **rioni**; those locations were then filtered so that those with more than 50 italian restaurants in radius of 500m and those with a distance greater then 1Km from the Rome Center were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. 

Result of all this is 3 zones containing largest number of potential new restaurant locations based on number of  existing venues. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! 

Purpose of this analysis was to only provide info on areas close to Rome center but not crowded with existing restaurants (particularly Italian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. 

Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.


## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Rome areas close to center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. 

By calculating restaurant density distribution from Foursquare data we generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. 

Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.
